<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_1/Hack_1_Image_Search/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 1 Hackathon - Image Search

# Explore Datasets

In [37]:
import cv2
import numpy as np

In [38]:
def refine_image (image) :

    width = 336
    height = 336

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (width , height))
    image = np.array(image)[: , : , 0 : 3]

    return image

In [39]:
refined_image_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'

for i in range(22) :

    image = cv2.imread(f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/queries/{i}.jpg')

    resized_image = refine_image(image)

    with open(f'{refined_image_path}/{i}.npy' , 'wb') as file :

        np.save(file , resized_image)

# CLIP VisionModel For Encode Image

In [40]:
!nvidia-smi

Sun May 26 12:54:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   58C    P0              30W /  72W |  15155MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [41]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import torch

In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [43]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [44]:
inputs = processor(images = image, return_tensors = "pt")
image_features = model.get_image_features(**inputs)

In [45]:
image_features.shape

torch.Size([1, 768])

# Solution-CLIP Zeroshot For Image Similarity Search

## Cosine Similarity

In [46]:
import numpy as np
from torch import nn

def cosine_similarity (A , B) :

    A = np.array(A)
    B = np.array(B)

    norm_A = np.linalg.norm(A)

    similarity_matrix = (A @ B.T).astype('float32') / norm_A

    for i in range(B.shape[0]) :

        norm_B = np.linalg.norm(B[i])

        similarity_matrix[0][i] /= norm_B

    similarity_matrix = similarity_matrix.reshape(-1)

    # softmax
    # similarity_matrix = np.exp(similarity_matrix)/sum(np.exp(similarity_matrix))

    target_class = similarity_matrix.argmax(axis = 0)
    maximum_similarity = similarity_matrix[target_class]

    return maximum_similarity , target_class

In [47]:
print(cosine_similarity(np.array([[1 , 2  ,3]]) , np.array([[1 , 2 , 3] , [3 , 4 , 5]])))

(0.99999994, 0)


## Define Model CLIP

In [48]:
from PIL import Image
import requests
from transformers import AutoProcessor , CLIPModel

## openai/clip-vit-large-patch14

In [49]:
import torch

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14').to(device)
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

## Read queries folder image and store to numpy array

In [51]:
import cv2
import os
import numpy as np

In [52]:
refined_queries_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'
images = []

for file_name in os.listdir(refined_queries_path) :

    with open(f'{refined_queries_path}/{file_name}' , 'rb') as file :

        image = np.load(file)

    images.append(image)

inputs = processor(images = images , return_tensors = 'pt').to(device)
image_features = model.get_image_features(** inputs)

In [53]:
image_features = image_features.cpu().detach().numpy()

In [54]:
with open('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'wb') as file :

    np.save(file , image_features)


## Read test folder images to inference

In [55]:
from tqdm import tqdm
import pandas as pd

In [56]:
with open('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'rb') as file :

    queries_matrix = np.load(file)

In [57]:
queries_matrix.shape

(22, 768)

In [58]:
submissions = {
    'img_file' : [] ,
    'class' : []
}

In [59]:
threshold = 0

In [60]:
test_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    threshold += maximum_similarity

100%|██████████| 1120/1120 [00:49<00:00, 22.52it/s]


In [61]:
threshold /= 1120
threshold

0.8227814406688724

In [62]:
test_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    if maximum_similarity < threshold :

        target_class = 22

    submissions['img_file'].append(file_name)
    submissions['class'].append(target_class)

100%|██████████| 1120/1120 [00:42<00:00, 26.34it/s]


In [63]:
submissions = pd.DataFrame.from_dict(submissions)
submissions_path = '/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submissions.csv'

with open(submissions_path, 'w') as csv_file:

    submissions.to_csv(path_or_buf = csv_file , index = False)

In [66]:
submissions.groupby('class').count()

,img_file
class,
0,4
1,2
2,114
3,41
4,4
5,63
6,24
7,2
8,1
